In [1]:
import random
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation,Dense,Conv2D,MaxPool2D,Flatten,BatchNormalization,Reshape
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import itertools

In [4]:
import os
#os.chdir('data')
print(os.getcwd())

c:\Users\HP\Documents\GitHub\CNN-ON-FPGA-MSc.-\Training Model(Python)\data


In [5]:
import shutil
shutil.move(f'cloudy','Train')
shutil.move(f'desert','Train')
shutil.move(f'green_area','Train')
shutil.move(f'water','Train')





'Train\\water'

In [6]:
os.mkdir(f'Validate/cloudy')
os.mkdir(f'Validate/desert')
os.mkdir(f'Validate/green_area')
os.mkdir(f'Validate/water')
os.mkdir(f'Test/cloudy')
os.mkdir(f'Test/desert')
os.mkdir(f'Test/green_area')
os.mkdir(f'Test/water')


In [7]:
os.mkdir(f'deletebin')



In [15]:
import os

# Set the path to the 'cloudy' directory
cwd= os.getcwd()
directory = f'{cwd}\\Train\\cloudy'

# Get a list of all files in the directory
file_list = os.listdir(directory)

# Count the number of files in the directory
file_count = len([file for file in file_list if os.path.isfile(os.path.join(directory, file))])

print(f'There are {file_count} files in the "cloudy" directory.')


There are 1500 files in the "cloudy" directory.


In [18]:
train_path='Train'
test_path='Test'
valid_path='Validate'

In [16]:

valid_samples1=random.sample(os.listdir(f'Train/cloudy'),50)
for j in valid_samples1:
    shutil.move(f'Train/cloudy/{j}',f'Validate/cloudy')

valid_samples2=random.sample(os.listdir(f'Train/water'),50)
for j in valid_samples2:
    shutil.move(f'Train/water/{j}',f'Validate/water')

valid_samples3=random.sample(os.listdir(f'Train/desert'),50)
for j in valid_samples3:
    shutil.move(f'Train/desert/{j}',f'Validate/desert')
valid_samples4=random.sample(os.listdir(f'Train/green_area'),50)
for j in valid_samples4:
    shutil.move(f'Train/green_area/{j}',f'Validate/green_area')



In [17]:

test_samples1=random.sample(os.listdir(f'Train/cloudy'),50)
for j in test_samples1:
    shutil.move(f'Train/cloudy/{j}',f'Test/cloudy')

test_samples2=random.sample(os.listdir(f'Train/water'),50)
for j in test_samples2:
    shutil.move(f'Train/water/{j}',f'Test/water')

test_samples3=random.sample(os.listdir(f'Train/desert'),50)
for j in test_samples3:
    shutil.move(f'Train/desert/{j}',f'Test/desert')
test_samples4=random.sample(os.listdir(f'Train/green_area'),50)
for j in test_samples4:
    shutil.move(f'Train/green_area/{j}',f'Test/green_area')

In [19]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=train_path,target_size=(256,256),batch_size=10)
valid_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=valid_path,target_size=(256,256),batch_size=10)
test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=test_path,target_size=(256,256),batch_size=10,shuffle=False)


Found 5231 images belonging to 4 classes.
Found 200 images belonging to 4 classes.
Found 200 images belonging to 4 classes.


In [20]:
mobile=tf.keras.applications.mobilenet.MobileNet()


In [21]:
mobile.summary()

Model: "mobilenet_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 112, 112, 32)   │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 112, 112, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 112, 112, 32)   │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 112, 112, 64)   │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 56, 56, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 56, 56, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 56, 56, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 4,253,864 (16.23 MB)

 Trainable params: 4,231,976 (16.14 MB)

 Non-trainable params: 21,888 (85.50 KB)